In [1]:
!git clone https://gitlab.com/bigirqu/quranqa.git
!pip install git+https://github.com/neuml/txtai datasets pandas
!pip install farasapy

Cloning into 'quranqa'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 247 (delta 29), reused 71 (delta 22), pack-reused 163
Receiving objects: 100% (247/247), 241.68 KiB | 5.49 MiB/s, done.
Resolving deltas: 100% (83/83), done.
  Cloning https://github.com/neuml/txtai to /tmp/pip-req-build-1v36y5rh
  Running command git clone -q https://github.com/neuml/txtai /tmp/pip-req-build-1v36y5rh
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 325 kB 33.7 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 3.8 MB 52.6 MB/s 
     |████████████████████████████████| 6.5 MB 12.3 MB/s 
     |████████████████████████████████| 8.6 MB 25.6 MB/s 
     |████████████████████████████████| 895 kB 48.7 MB/s 
     |█████████████████████████

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import sys
sys.path.insert(1, './quranqa/code')
import read_write_qrcd
import quranqa22_eval

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from txtai.pipeline import HFTrainer
import torch, string, re, os, json
import operator

from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.diacratizer import FarasaDiacritizer 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer



In [ ]:

train_passage_question_objects = read_write_qrcd.read_JSONL_file('./quranqa/datasets/qrcd_v1.1_train.jsonl')
dev_passage_question_objects = read_write_qrcd.read_JSONL_file('./quranqa/datasets/qrcd_v1.1_dev.jsonl')


In [ ]:

train_data = [dict({"pq_id": passage_question_object.pq_id,
                    "question": quranqa22_eval.normalize_text(passage_question_object.question),
                    "context": passage_question_object.passage, 
                    "answers": r.text})
              for passage_question_object in train_passage_question_objects
              for r in passage_question_object.answers]


dev_data = [dict({"pq_id": passage_question_object.pq_id,
                  "question": quranqa22_eval.normalize_text(passage_question_object.question),
                  "context": passage_question_object.passage, 
                 }) 
            for passage_question_object in dev_passage_question_objects
            for r in passage_question_object.answers]



In [ ]:
pretrained_models = ["wissamantoun/araelectra-base-artydiqa", #num_train_epochs=2
                     "salti/AraElectra-base-finetuned-ARCD", #num_train_epochs=8
                     #"gfdgdfgdg/arap_qa_bert_large_v2",
                     #"gfdgdfgdg/arap_qa_bert_v2",
                     #"gfdgdfgdg/arap_qa_bert"
                     ]


def tokenize_function(row,tokenizer):
    return tokenizer.encode_plus(row['question'], row['context'], 
                                 return_tensors='pt', padding=True, 
                                 truncation=True,max_length=512, 
                                 add_special_tokens = True)

topk_n = 20
def get_prediction(row, model, tokenizer):
    inputs = tokenize_function(row,tokenizer)
    output = model(**inputs)
    
    #print(torch.topk(output.start_logits.flatten(), topk_n))
    answer_starts_val = torch.topk(output.start_logits.flatten(), topk_n).values
    answer_ends_val = torch.topk(output.end_logits.flatten(), topk_n).values
    answer_starts = torch.topk(output.start_logits.flatten(), topk_n).indices
    answer_ends = torch.topk(output.end_logits.flatten(), topk_n).indices
    
    answers = []
    rank = 1
    for answer_start,answer_end,answer_start_v,answer_end_v in zip(answer_starts,answer_ends,answer_starts_val,answer_ends_val):
        answer = {}
        ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end+1]))
        if len(ans.strip())>0:
            answer["answer"] = ans
            answer["rank"] = rank
            answer["score"] = answer_start_v.item()+answer_end_v.item()
            rank = rank +1
            answers.append(answer)
        if rank == 6:
            break
        #answer = str(answer_start_v.item())+' '+ answer +' '+ str(answer_end_v.item())
    return answers


In [ ]:

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    print(tokenizer.decode(prediction[0]))
    print(tokenizer.decode(prediction))
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    exact_match = compute_exact_match(pred, labels)
    f1 = compute_f1(pred, labels)

    return { "f1": f1, "exact_match": exact_match}


In [ ]:

def Fine_tune(model_name,traindata,devdata,num_train_epochs,batch_size):
    trainer = HFTrainer()

    return trainer(model_name, traindata,#devdata,
                   task="question-answering",
                   metric_for_best_model = 'f1',
                   #evaluation_strategy ='epoch',
                   #eval_steps = 500, # Evaluation and Save happens every 50 steps
                   #save_total_limit = 15,
                   save_strategy='no',
                   learning_rate=2e-5,
                   per_device_train_batch_size=batch_size,
                   #per_device_eval_batch_size=batch_size,
                   num_train_epochs=num_train_epochs,
                   weight_decay=0.01,
                   #load_best_model_at_end=False, # change this to True after hyperparameter tuning
                   )
        

    #return trainer(model_name, data, task="question-answering", num_train_epochs=num_train_epochs)#num_train_epochs=10

In [ ]:
def test_model(tokenizer,model,data):
    result = {}
    for row in data:
        pred = get_prediction(row, model, tokenizer)
        result[row["pq_id"]] = pred
    return result

In [ ]:
import ast
os.makedirs('Fine_tuned')
SAVEING_PATH = '/content/drive/MyDrive/arap_qa_bert_large_v2'
#os.makedirs(SAVEING_PATH)
num_train_epochs=7
batch_size=3
best_metrics = {'pRR': 0.5427106814488388, 'exact_match': 0.25688073394495414, 'f1': 0.498117046310898}


model_name = "gfdgdfgdg/arap_qa_bert_large_v2" #num_train_epochs=2
for num_train_epochs in range(21,30):
    model, tokenizer = Fine_tune(model_name,train_data,dev_data,num_train_epochs,batch_size)
    model.save_pretrained('Fine_tuned')
    tokenizer.save_pretrained('Fine_tuned')
    tokenizer = AutoTokenizer.from_pretrained('./Fine_tuned')
    model = AutoModelForQuestionAnswering.from_pretrained('./Fine_tuned')
    result = test_model(tokenizer,model,dev_data)
    open('./quranqa/datasets/wissam_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
    metrics = !python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'
    metrics = ast.literal_eval(metrics[-1])
    print(num_train_epochs,num_train_epochs,num_train_epochs,num_train_epochs)
    print(metrics)
    if best_metrics == '' or metrics["pRR"] > best_metrics["pRR"]:
      best_metrics = metrics
      !\cp Fine_tuned/* $SAVEING_PATH
      print('saved\n\n')


Downloading:   0%|          | 0.00/657 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Step,Training Loss
500,2.164400
1000,1.172700
1500,0.888200
2000,0.718500
2500,0.567300
3000,0.536500
3500,0.530200
4000,0.464200
4500,0.419000
5000,0.433000


24 24 24 24
{'pRR': 0.5204405556960493, 'exact_match': 0.26605504587155965, 'f1': 0.48350374501917337}


Step,Training Loss
500,2.151000
1000,1.103700
1500,0.849100
2000,0.726100
2500,0.597500
3000,0.534500
3500,0.520400
4000,0.509000
4500,0.433400
5000,0.449600


25 25 25 25
{'pRR': 0.5907354179653759, 'exact_match': 0.3119266055045872, 'f1': 0.5663697761252415}
saved




Step,Training Loss
500,2.168400
1000,1.157000
1500,0.860700
2000,0.716100
2500,0.584000
3000,0.512100
3500,0.485300
4000,0.447900
4500,0.398200
